In [7]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
from scipy import optimize
import pandas as pd
import glob
import math
import os

import multiprocessing
from multiprocessing import Pool

In [8]:
# Specifying simulation directory and the directory to save results in
wdir = str(input('Enter simulation directory path: '))

# Specifying a snapshot for temporal analysis
sdir = wdir + 'temporal_analysis/'

Enter simulation directory path: /Users/thepoetoftwilight/Documents/CASSI2020/CASSI2020-Code+Results/Results/m12i_res7100/


In [9]:
def plot_snap(info):
    
    snap = info[0]
    redshift = info[1]
    metals = info[2]
    spath_metals = info[3]
    x_min = info[4]
    x_max = info[5]
    y_min = info[6]
    y_max = info[7]
    
    for m in metals:
    
        num_df = pd.read_csv(spath_metals[m] + 'data/num/{0}-num_{1}_data.csv'.format(str(snap), m))
        fit_df = pd.read_csv(spath_metals[m] + 'data/fit/{0}-fit_{1}_data.csv'.format(str(snap), m))
        
        centers = num_df['abundance'].to_list()
        heights = num_df['num_val'].to_list()
        fit = fit_df['fit_val'].to_list()
    
        # Create plots for the numerical PDF and store them
        fig, ax = plt.subplots(figsize = (15, 13))

        if(np.min(centers) > x_min):
            empty_left = [x_min, np.min(centers)]
            zeros = [0, 0]
            ax.plot(empty_left, zeros, lw = 5, color = 'orange')

        if(np.max(centers) < x_max):
            empty_right = [np.max(centers), x_max]
            zeros = [0, 0]
            ax.plot(empty_right, zeros, lw = 5, color = 'orange')

        ax.plot(centers, heights, lw = 5, color = 'orange')

        ax.set_xlabel(
            r'$\left[\frac{{{0}}}{{H}} \right]$'.format(m.title()), 
            fontsize = 36, labelpad = 5)
        ax.set_ylabel(r'$p_{{{0}, X}} \left( \left[ \frac{{{0}}}{{H}} \right] \right)$'.format(m.title()),
             fontsize = 38, labelpad = 10)

        ax.set_title('z = {}'.format(str(round(redshift, 2))), y = 1.04)
        ax.ticklabel_format(axis='both', style='sci', scilimits=(0,0))
        
        ax.set_xlim(xmin = x_min, xmax = x_max)
        ax.set_ylim(ymin = y_min, ymax = y_max)
        
        ax.tick_params(labelsize = 36, pad = 10)
        ax.yaxis.offsetText.set_fontsize(36)
        
        fig.savefig(spath_metals[m] + 'images/num/' + 
                    '{0}-num_{1}.png'.format(str(snap), m.title()))
        plt.close()

        # Create plots for the fitted PDF and store them
        fig, ax = plt.subplots(figsize = (15, 13))

        if(np.min(centers) > x_min):
            empty_left = [x_min, np.min(centers)]
            zeros = [0, 0]
            ax.plot(empty_left, zeros, lw = 5, color = 'orange')

        if(np.max(centers) < x_max):
            empty_right = [np.max(centers), x_max]
            zeros = [0, 0]
            ax.plot(empty_right, zeros, lw = 5, color = 'orange')

        ax.plot(centers, heights, label = 'Simulation', lw = 5, color = 'orange')
        ax.plot(centers, fit, label = 'Analytic Fit', lw = 5, color = 'orange', linestyle = ':')

        ax.set_xlabel(
            '[{}/H]'.format(m.title()), 
            fontsize = 38, labelpad = 5)
        ax.set_ylabel('p([{}/H])'.format(m.title()),
             fontsize = 38, labelpad = 10)

        ax.set_title('z = {}'.format(str(round(redshift, 2))), y = 1.04, fontsize = 38)
        ax.ticklabel_format(axis='both', style='sci', scilimits=(0,0))
        
        ax.tick_params(labelsize = 38, pad = 10)
        ax.yaxis.offsetText.set_fontsize(36)
        
        ax.set_xlim(xmin = x_min, xmax = x_max)
        ax.set_ylim(ymin = y_min, ymax = y_max)
        
        ax.legend(prop={'size': 38}, loc = 'upper right')
        fig.savefig(spath_metals[m] + 'images/fit/' + 
                    '{0}-fit_{1}.png'.format(str(snap), m.title()))
        
        plt.close()
        
        print('Plotted snapshot {0} for {1}'.format(str(snap), m.title()))

In [10]:
# Get all rendered indices

rendered_df = pd.read_csv(sdir + 'rendered_snap_stats.csv')
rendered_indices = rendered_df['snap'].to_list()
redshifts = rendered_df['redshift'].to_list()
num_fits = len(rendered_indices)

# Get rendered metals

metal_df = pd.read_csv((sdir + 'metal_list.csv'))
metals = metal_df['metals'].to_list()

# Create a list of paths for all metals
spath_metals = {}

for m in metals:
    spath_metals[m] = sdir + m + '/'

In [11]:
# Find universal x_lim and y_lim across all snapshots

# Initializing 

x_min = math.inf
x_max = -math.inf
y_min = math.inf
y_max = -math.inf

# Iterating across all metals and all snapshots to find universal plot bounds

for m in metals:
    
    for index in rendered_indices:
        
        num_df = pd.read_csv(spath_metals[m] + 'data/num/{0}-num_{1}_data.csv'.format(str(index), m))
        
        centers = num_df['abundance'].to_list()
        num_vals = num_df['num_val'].to_list()
        
        curr_x_min = np.min(centers)
        curr_x_max = np.max(centers)
        curr_y_min = np.min(num_vals)
        curr_y_max = np.max(num_vals)
        
        if(curr_x_min < x_min):
            x_min = curr_x_min
            
        if(curr_x_max > x_max):
            x_max = curr_x_max
        
        if(curr_y_min < y_min):
            y_min = curr_y_min
            
        if(curr_y_max > y_max):
            y_max = curr_y_max

In [12]:
# Make data arrays for each snapshot
    
if __name__ == '__main__':
    
    n_proc = multiprocessing.cpu_count()
        
    with Pool(processes = n_proc) as pool:
        
        pool.map(plot_snap, [(rendered_indices[i], redshifts[i], metals, spath_metals, x_min, x_max,
                             y_min, y_max) for i in range(0, num_fits)])

Plotted snapshot 173
Plotted snapshot 140
Plotted snapshot 107
Plotted snapshot 68
Plotted snapshot 140
Plotted snapshot 173
Plotted snapshot 68
Plotted snapshot 107
Plotted snapshot 173
Plotted snapshot 140
Plotted snapshot 68
Plotted snapshot 107
Plotted snapshot 173
Plotted snapshot 140
Plotted snapshot 68
Plotted snapshot 107
Plotted snapshot 173
Plotted snapshot 68
Plotted snapshot 140
Plotted snapshot 107
Plotted snapshot 173
Plotted snapshot 140
Plotted snapshot 68
Plotted snapshot 107
Plotted snapshot 173
Plotted snapshot 68
Plotted snapshot 140
Plotted snapshot 107
Plotted snapshot 173
Plotted snapshot 68
Plotted snapshot 140
Plotted snapshot 107
Plotted snapshot 173
Plotted snapshot 68
Plotted snapshot 140
Plotted snapshot 107
Plotted snapshot 174
Plotted snapshot 141
Plotted snapshot 69
Plotted snapshot 108
Plotted snapshot 174
Plotted snapshot 69
Plotted snapshot 141
Plotted snapshot 108
Plotted snapshot 174
Plotted snapshot 69
Plotted snapshot 141
Plotted snapshot 108
Plot

Plotted snapshot 82
Plotted snapshot 117
Plotted snapshot 184
Plotted snapshot 151
Plotted snapshot 82
Plotted snapshot 118
Plotted snapshot 184
Plotted snapshot 151
Plotted snapshot 82
Plotted snapshot 118
Plotted snapshot 184
Plotted snapshot 151
Plotted snapshot 82
Plotted snapshot 118
Plotted snapshot 184
Plotted snapshot 151
Plotted snapshot 82
Plotted snapshot 118
Plotted snapshot 184
Plotted snapshot 151
Plotted snapshot 82
Plotted snapshot 118
Plotted snapshot 184
Plotted snapshot 151
Plotted snapshot 82
Plotted snapshot 118
Plotted snapshot 184
Plotted snapshot 151
Plotted snapshot 82
Plotted snapshot 118
Plotted snapshot 184
Plotted snapshot 151
Plotted snapshot 82
Plotted snapshot 118
Plotted snapshot 184
Plotted snapshot 151
Plotted snapshot 118
Plotted snapshot 185
Plotted snapshot 152
Plotted snapshot 83
Plotted snapshot 185
Plotted snapshot 152
Plotted snapshot 83
Plotted snapshot 119
Plotted snapshot 185
Plotted snapshot 83
Plotted snapshot 119
Plotted snapshot 185
Plot

Plotted snapshot 128
Plotted snapshot 195
Plotted snapshot 162
Plotted snapshot 128
Plotted snapshot 195
Plotted snapshot 162
Plotted snapshot 195
Plotted snapshot 96
Plotted snapshot 195
Plotted snapshot 96
Plotted snapshot 129
Plotted snapshot 195
Plotted snapshot 162
Plotted snapshot 96
Plotted snapshot 129
Plotted snapshot 162
Plotted snapshot 96
Plotted snapshot 129
Plotted snapshot 162
Plotted snapshot 96
Plotted snapshot 129
Plotted snapshot 162
Plotted snapshot 195
Plotted snapshot 96
Plotted snapshot 129
Plotted snapshot 162
Plotted snapshot 195
Plotted snapshot 96
Plotted snapshot 129
Plotted snapshot 162
Plotted snapshot 195
Plotted snapshot 96
Plotted snapshot 129
Plotted snapshot 162
Plotted snapshot 96
Plotted snapshot 195
Plotted snapshot 129
Plotted snapshot 163
Plotted snapshot 97
Plotted snapshot 196
Plotted snapshot 129
Plotted snapshot 163
Plotted snapshot 196
Plotted snapshot 97
Plotted snapshot 130
Plotted snapshot 163
Plotted snapshot 196
Plotted snapshot 97
Plot

Plotted snapshot 139
Plotted snapshot 172
Plotted snapshot 205
Plotted snapshot 106
Plotted snapshot 139
Plotted snapshot 206
Plotted snapshot 239
Plotted snapshot 272
Plotted snapshot 139
Plotted snapshot 206
Plotted snapshot 239
Plotted snapshot 272
Plotted snapshot 305
Plotted snapshot 206
Plotted snapshot 239
Plotted snapshot 272
Plotted snapshot 305
Plotted snapshot 206
Plotted snapshot 272
Plotted snapshot 239
Plotted snapshot 305
Plotted snapshot 206
Plotted snapshot 272
Plotted snapshot 239
Plotted snapshot 305
Plotted snapshot 206
Plotted snapshot 272
Plotted snapshot 239
Plotted snapshot 305
Plotted snapshot 206
Plotted snapshot 272
Plotted snapshot 239
Plotted snapshot 305
Plotted snapshot 206
Plotted snapshot 272
Plotted snapshot 305
Plotted snapshot 239
Plotted snapshot 206
Plotted snapshot 272
Plotted snapshot 305
Plotted snapshot 239
Plotted snapshot 207
Plotted snapshot 273
Plotted snapshot 305
Plotted snapshot 240
Plotted snapshot 207
Plotted snapshot 273
Plotted snaps

Plotted snapshot 249
Plotted snapshot 216
Plotted snapshot 315
Plotted snapshot 282
Plotted snapshot 249
Plotted snapshot 216
Plotted snapshot 315
Plotted snapshot 283
Plotted snapshot 249
Plotted snapshot 217
Plotted snapshot 315
Plotted snapshot 283
Plotted snapshot 250
Plotted snapshot 217
Plotted snapshot 315
Plotted snapshot 283
Plotted snapshot 250
Plotted snapshot 217
Plotted snapshot 315
Plotted snapshot 283
Plotted snapshot 250
Plotted snapshot 217
Plotted snapshot 316
Plotted snapshot 283
Plotted snapshot 250
Plotted snapshot 217
Plotted snapshot 316
Plotted snapshot 283
Plotted snapshot 250
Plotted snapshot 217
Plotted snapshot 316
Plotted snapshot 283
Plotted snapshot 250
Plotted snapshot 217
Plotted snapshot 316
Plotted snapshot 283
Plotted snapshot 250
Plotted snapshot 217
Plotted snapshot 283
Plotted snapshot 316
Plotted snapshot 250
Plotted snapshot 217
Plotted snapshot 284
Plotted snapshot 316
Plotted snapshot 250
Plotted snapshot 284
Plotted snapshot 218
Plotted snaps

...

Plotted snapshot 392
Plotted snapshot 457
Plotted snapshot 425
Plotted snapshot 359
Plotted snapshot 392
Plotted snapshot 457
Plotted snapshot 425
Plotted snapshot 359
Plotted snapshot 392
Plotted snapshot 458
Plotted snapshot 425
Plotted snapshot 359
Plotted snapshot 392
Plotted snapshot 458
Plotted snapshot 425
Plotted snapshot 359
Plotted snapshot 392
Plotted snapshot 458
Plotted snapshot 425
Plotted snapshot 359
Plotted snapshot 392
Plotted snapshot 458
Plotted snapshot 425
Plotted snapshot 359
Plotted snapshot 392
Plotted snapshot 458
Plotted snapshot 425
Plotted snapshot 360
Plotted snapshot 393
Plotted snapshot 458
Plotted snapshot 426
Plotted snapshot 360
Plotted snapshot 393
Plotted snapshot 458
Plotted snapshot 426
Plotted snapshot 360
Plotted snapshot 393
Plotted snapshot 458
Plotted snapshot 426
Plotted snapshot 360
Plotted snapshot 393
Plotted snapshot 458
Plotted snapshot 426
Plotted snapshot 360
Plotted snapshot 393
Plotted snapshot 459
Plotted snapshot 426
Plotted snaps

Plotted snapshot 370
Plotted snapshot 403
Plotted snapshot 436
Plotted snapshot 468
Plotted snapshot 370
Plotted snapshot 403
Plotted snapshot 436
Plotted snapshot 370
Plotted snapshot 468
Plotted snapshot 403
Plotted snapshot 436
Plotted snapshot 468
Plotted snapshot 370
Plotted snapshot 403
Plotted snapshot 436
Plotted snapshot 469
Plotted snapshot 370
Plotted snapshot 403
Plotted snapshot 436
Plotted snapshot 469
Plotted snapshot 370
Plotted snapshot 403
Plotted snapshot 436
Plotted snapshot 469
Plotted snapshot 370
Plotted snapshot 403
Plotted snapshot 436
Plotted snapshot 469
Plotted snapshot 370
Plotted snapshot 403
Plotted snapshot 436
Plotted snapshot 469
Plotted snapshot 470
Plotted snapshot 503
Plotted snapshot 536
Plotted snapshot 469
Plotted snapshot 470
Plotted snapshot 503
Plotted snapshot 536
Plotted snapshot 469
Plotted snapshot 470
Plotted snapshot 503
Plotted snapshot 536
Plotted snapshot 469
Plotted snapshot 470
Plotted snapshot 503
Plotted snapshot 536
Plotted snaps

Plotted snapshot 480
Plotted snapshot 545
Plotted snapshot 513
Plotted snapshot 578
Plotted snapshot 480
Plotted snapshot 546
Plotted snapshot 513
Plotted snapshot 578
Plotted snapshot 480
Plotted snapshot 546
Plotted snapshot 513
Plotted snapshot 578
Plotted snapshot 480
Plotted snapshot 546
Plotted snapshot 513
Plotted snapshot 578
Plotted snapshot 480
Plotted snapshot 546
Plotted snapshot 513
Plotted snapshot 579
Plotted snapshot 480
Plotted snapshot 546
Plotted snapshot 513
Plotted snapshot 579
Plotted snapshot 480
Plotted snapshot 546
Plotted snapshot 513
Plotted snapshot 579
Plotted snapshot 546
Plotted snapshot 480
Plotted snapshot 513
Plotted snapshot 579
Plotted snapshot 546
Plotted snapshot 480
Plotted snapshot 513
Plotted snapshot 579
Plotted snapshot 481
Plotted snapshot 546
Plotted snapshot 514
Plotted snapshot 579
Plotted snapshot 481
Plotted snapshot 514
Plotted snapshot 547
Plotted snapshot 579
Plotted snapshot 481
Plotted snapshot 514
Plotted snapshot 547
Plotted snaps

Plotted snapshot 524
Plotted snapshot 557
Plotted snapshot 490
Plotted snapshot 589
Plotted snapshot 524
Plotted snapshot 557
Plotted snapshot 490
Plotted snapshot 524
Plotted snapshot 589
Plotted snapshot 557
Plotted snapshot 490
Plotted snapshot 589
Plotted snapshot 524
Plotted snapshot 557
Plotted snapshot 589
Plotted snapshot 490
Plotted snapshot 524
Plotted snapshot 557
Plotted snapshot 491
Plotted snapshot 524
Plotted snapshot 589
Plotted snapshot 557
Plotted snapshot 491
Plotted snapshot 589
Plotted snapshot 524
Plotted snapshot 558
Plotted snapshot 491
Plotted snapshot 589
Plotted snapshot 524
Plotted snapshot 558
Plotted snapshot 491
Plotted snapshot 589
Plotted snapshot 524
Plotted snapshot 558
Plotted snapshot 491
Plotted snapshot 589
Plotted snapshot 525
Plotted snapshot 558
Plotted snapshot 491
Plotted snapshot 525
Plotted snapshot 590
Plotted snapshot 558
Plotted snapshot 491
Plotted snapshot 590
Plotted snapshot 525
Plotted snapshot 558
Plotted snapshot 491
Plotted snaps

Plotted snapshot 568
Plotted snapshot 534
Plotted snapshot 599
Plotted snapshot 501
Plotted snapshot 568
Plotted snapshot 535
Plotted snapshot 599
Plotted snapshot 501
Plotted snapshot 568
Plotted snapshot 535
Plotted snapshot 501
Plotted snapshot 600
Plotted snapshot 568
Plotted snapshot 535
Plotted snapshot 501
Plotted snapshot 600
Plotted snapshot 568
Plotted snapshot 535
Plotted snapshot 600
Plotted snapshot 501
Plotted snapshot 568
Plotted snapshot 535
Plotted snapshot 600
Plotted snapshot 502
Plotted snapshot 568
Plotted snapshot 535
Plotted snapshot 600
Plotted snapshot 502
Plotted snapshot 535
Plotted snapshot 600
Plotted snapshot 502
Plotted snapshot 535
Plotted snapshot 502
Plotted snapshot 600
Plotted snapshot 535
Plotted snapshot 502
Plotted snapshot 600
Plotted snapshot 502
Plotted snapshot 600
Plotted snapshot 502
Plotted snapshot 502
Plotted snapshot 502
